In [1]:
import csv
import pprint

# with open("skill_levels.csv", encoding="utf8") as f:
#     skill_table = [x for x in csv.DictReader(f)]

with open("armor_base.csv", encoding="utf8") as f:
    armor_table = [x for x in csv.DictReader(f)]

with open("armor_skills_ext.csv", encoding="utf8") as f:
    armor_skills_table = dict((x["base_name_en"], x) for x in csv.DictReader(f))

with open("charm_base.csv", encoding="utf8") as f:
    charm_table = [x for x in csv.DictReader(f)]

with open("decoration_base.csv", encoding="utf8") as f:
    deco_table = [x for x in csv.DictReader(f)]

print(armor_table[-1])
print(armor_skills_table["Leather Headgear"])
print(charm_table[0])
print(deco_table[0])


{'id': '1470', 'name_en': 'Wyverian Ears α+', 'rarity': '9', 'type': 'head', 'gender': 'both', 'slot_1': '4', 'slot_2': '0', 'slot_3': '0', 'defense_base': '140', 'defense_max': '178', 'defense_augment_max': '198', 'defense_fire': '1', 'defense_water': '1', 'defense_thunder': '1', 'defense_ice': '1', 'defense_dragon': '5'}
{'base_name_en': 'Leather Headgear', 'skill1_name': 'Hunger Resistance', 'skill1_level': '1', 'skill2_name': '', 'skill2_level': ''}
{'id': '1', 'name_en': 'Poison Charm I', 'previous_en': '', 'rarity': '3', 'skill1_name': 'Poison Resistance', 'skill1_level': '1', 'skill2_name': '', 'skill2_level': ''}
{'id': '1', 'name_en': 'Antidote Jewel 1', 'slot': '1', 'rarity': '5', 'skill1_name': 'Poison Resistance', 'skill1_level': '1', 'skill2_name': '', 'skill2_level': '', 'icon_color': 'Violet'}


In [2]:
# join and reformat.
equipment_table = []
for armor in armor_table:
    if int(armor["rarity"]) < 8:
        continue
    equipment_table.append({
        "name_en": armor["name_en"],
        "rarity": int(armor["rarity"]),
        "type": armor["type"],
        "slot_1": int(armor["slot_1"]),
        "slot_2": int(armor["slot_2"]),
        "slot_3": int(armor["slot_3"]),
        "defense_base": int(armor["defense_base"]),
        "defense_max": int(armor["defense_max"]),
        "defense_augment_max": int(armor["defense_augment_max"]),
        # and whatever ig

        "skill1_name": armor_skills_table[armor["name_en"]]["skill1_name"],
        "skill1_level": int(armor_skills_table[armor["name_en"]]["skill1_level"]) if armor_skills_table[armor["name_en"]]["skill1_name"] != "" else 0,
        "skill2_name": armor_skills_table[armor["name_en"]]["skill2_name"],
        "skill2_level": int(armor_skills_table[armor["name_en"]]["skill2_level"]) if armor_skills_table[armor["name_en"]]["skill2_name"] != "" else 0,

        "1_slots": int(int(armor["slot_1"]) >= 1) + int(int(armor["slot_2"]) >= 1) + int(int(armor["slot_3"]) >= 1),
        "2_slots": int(int(armor["slot_1"]) >= 2) + int(int(armor["slot_2"]) >= 2) + int(int(armor["slot_3"]) >= 2),
        "3_slots": int(int(armor["slot_1"]) >= 3) + int(int(armor["slot_2"]) >= 3) + int(int(armor["slot_3"]) >= 3),
        "4_slots": int(int(armor["slot_1"]) >= 4) + int(int(armor["slot_2"]) >= 4) + int(int(armor["slot_3"]) >= 4),
    })
for charm in charm_table:
    equipment_table.append({
        "name_en": charm["name_en"],
        "rarity": int(charm["rarity"]),
        "type": "charm",
        "slot_1": 0,
        "slot_2": 0,
        "slot_3": 0,
        "defense_base": 0,
        "defense_max": 0,
        "defense_augment_max": 0,
        # and whatever ig

        "skill1_name": charm["skill1_name"],
        "skill1_level": int(charm["skill1_level"]) if charm["skill1_name"] != "" else 0,
        "skill2_name": charm["skill2_name"],
        "skill2_level": int(charm["skill2_level"]) if charm["skill2_name"] != "" else 0,
        
        "1_slots": 0,
        "2_slots": 0,
        "3_slots": 0,
        "4_slots": 0,
    })
for deco in deco_table:
    equipment_table.append({
        "name_en": deco["name_en"],
        "rarity": int(deco["rarity"]),
        "type": "deco",
        "slot_1": -int(deco["slot"]), # hack. makes sense semantically.
        "slot_2": 0,
        "slot_3": 0,
        "defense_base": 0,
        "defense_max": 0,
        "defense_augment_max": 0,
        # and whatever ig

        "skill1_name": deco["skill1_name"],
        "skill1_level": int(deco["skill1_level"]) if deco["skill1_name"] != "" else 0,
        "skill2_name": deco["skill2_name"],
        "skill2_level": int(deco["skill2_level"]) if deco["skill2_name"] != "" else 0,

        "1_slots": -1 * int(int(deco["slot"]) >= 1), # hack. makes sense semantically.
        "2_slots": -1 * int(int(deco["slot"]) >= 2),
        "3_slots": -1 * int(int(deco["slot"]) >= 3),
        "4_slots": -1 * int(int(deco["slot"]) >= 4),
    })

In [3]:
skill_list = tuple(sorted(set(armor["skill1_name"] for armor in armor_skills_table.values() if armor["skill1_name"] != "").union(set(armor["skill2_name"] for armor in armor_skills_table.values() if armor["skill2_name"] != ""))))
for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
    out = tuple(skill for skill in skill_list if skill[0] == letter)
    if len(out) > 0:
        print(out)

('Affinity Sliding', 'Agitator', 'Airborne', 'Aquatic/Polar Mobility', 'Artillery', 'Attack Boost')
('BBQ Master', 'Blast Attack', 'Blast Resistance', 'Bleeding Resistance', 'Blight Resistance', 'Blindsider', 'Bombardier', 'Botanist')
('Capacity Boost', 'Carving Pro', 'Cliffhanger', 'Coalescence', 'Constitution', 'Critical Boost', 'Critical Draw', 'Critical Eye')
('Defense Boost', 'Detector', 'Divine Blessing', 'Dragon Attack', 'Dragon Resistance', 'Dungmaster')
('Earplugs', 'Effluvia Resistance', 'Effluvial Expert', 'Elderseal Boost', 'Entomologist', 'Evade Extender', 'Evade Window')
('Fire Attack', 'Fire Resistance', 'Flinch Free', 'Focus', "Forager's Luck", 'Fortify', 'Free Elem/Ammo Up', 'Free Meal')
('Geologist', 'Guard')
('Handicraft', 'Health Boost', 'Heat Guard', 'Heavy Artillery', 'Heroics', 'Honey Hunter', 'Horn Maestro', 'Hunger Resistance')
('Ice Attack', 'Ice Resistance', 'Intimidator', 'Iron Skin', 'Item Prolonger')
('Jump Master',)
('Latent Power', 'Leap of Faith')
('Mar

In [4]:
deco_list = tuple(sorted(set(deco["name_en"] for deco in deco_table)))
for level in range(1, 5):
    for letters in ("ABCDE", "FGHIJ", "KLMNO", "PQRST", "UVWXYZ"):
        out = tuple(deco for deco in deco_list if deco[0] in letters and int(deco[-1]) == level)
        if len(out) > 0:
            print(out)

('Antiblast Jewel 1', 'Antidote Jewel 1', 'Antipara Jewel 1', 'Artillery Jewel 1', 'Attack Jewel 1', 'Blast Jewel 1', 'Blaze Jewel 1', 'Bolt Jewel 1', 'Bomber Jewel 1', 'Botany Jewel 1', 'Crisis Jewel 1', 'Def Lock Jewel 1', 'Defense Jewel 1', 'Dragon Jewel 1', 'Dragon Res Jewel 1', 'Drain Jewel 1', 'Enduring Jewel 1', 'Expert Jewel 1')
('Fire Res Jewel 1', 'Fortitude Jewel 1', 'Friendship Jewel 1', 'Frost Jewel 1', 'Fungiform Jewel 1', 'Geology Jewel 1', 'Gobbler Jewel 1', 'Grinder Jewel 1', 'Heavy Artillery Jewel 1', 'Hungerless Jewel 1', 'Ice Res Jewel 1', 'Intimidator Jewel 1', 'Ironwall Jewel 1')
('Maintenance Jewel 1', 'Medicine Jewel 1', 'Meowster Jewel 1', 'Miasma Jewel 1', 'Mirewalker Jewel 1')
('Paralyzer Jewel 1', 'Pep Jewel 1', 'Protection Jewel 1', 'Recovery Jewel 1', 'Resistor Jewel 1', 'Satiated Jewel 1', 'Scent Jewel 1', 'Sheath Jewel 1', 'Sleep Jewel 1', 'Sonorous Jewel 1', 'Specimen Jewel 1', 'Steadfast Jewel 1', 'Stonethrower Jewel 1', 'Stream Jewel 1', 'Survival Jew

In [5]:
desired_skills = {
    "Artillery": 3,
    "Capacity Boost": 1,
    "Evade Extender": 3,
    "Quick Sheath": 3,
    "Divine Blessing": 3,
    "Slinger Capacity": 3,
    "Health Boost": 3,
    "Free Meal": 1,
}

owned_decos = {
    "Wind Resist/Protection Jewel 4": 1,
    "Brace/Vitality Jewel 4": 1,
    "Jumping/Physique Jewel 4": 1,
    "Stonethrower/Evasion Jewel 4": 1,
    "Earplug Jewel 3": 1,
    "Evasion Jewel 2": 1,
    "Sprinter Jewel 2": 1,
    "Warming Jewel 2": 1,
    "Vitality Jewel 1": 1000,
    "Friendship Jewel 1": 3,
    "Steadfast Jewel 1": 1000,
    "Sheath Jewel 1": 1,
    "Stonethrower Jewel 1": 3,
}
# to ignore, uncomment. but tbh itll just shove everything into decos.
# owned_decos = dict((x, 1000) for x in deco_list)

assert(all(desired in skill_list for desired in desired_skills))
assert(all(owned in deco_list for owned in owned_decos))

In [6]:
import numpy

# maximize this
defense_row = numpy.fromiter((equipment["defense_base"] for equipment in equipment_table), int)

# there must be exactly one of each
head_row = numpy.fromiter((equipment["type"] == "head" for equipment in equipment_table), int)
chest_row = numpy.fromiter((equipment["type"] == "chest" for equipment in equipment_table), int)
arms_row = numpy.fromiter((equipment["type"] == "arms" for equipment in equipment_table), int)
waist_row = numpy.fromiter((equipment["type"] == "waist" for equipment in equipment_table), int)
legs_row = numpy.fromiter((equipment["type"] == "legs" for equipment in equipment_table), int)
charm_row = numpy.fromiter((equipment["type"] == "charm" for equipment in equipment_table), int)

equipment_matrix = numpy.vstack((head_row, chest_row, arms_row, waist_row, legs_row, charm_row))
equipment_upper_bound = numpy.ones(6)
equipment_lower_bound = numpy.ones(6)

# unused slots cannot be negative. (there cannot be more decos than there are slots)
# by setup, a 3 slot deco takes a 3 slot spot, a 2 slot spot, and a 1 slot spot.
# if the armor had a 4 slot, its left untaken, but it would be unusable without the 3 slot, the 2 slot, the 1 slot.
# describe this nicer.
unused_4_slots = numpy.fromiter((equipment["4_slots"] for equipment in equipment_table), int)
unused_3_slots = numpy.fromiter((equipment["3_slots"] for equipment in equipment_table), int)
unused_2_slots = numpy.fromiter((equipment["2_slots"] for equipment in equipment_table), int)
unused_1_slots = numpy.fromiter((equipment["1_slots"] for equipment in equipment_table), int)
deco_slot_matrix = numpy.vstack((unused_4_slots, unused_3_slots, unused_2_slots, unused_1_slots))
deco_slot_lower_bound = numpy.zeros(4)

# there must be the user selected skills
def get_skill_level(equipment, skill):
    if equipment["skill1_name"] == skill:
        return equipment["skill1_level"]
    if equipment["skill2_name"] == skill:
        return equipment["skill2_level"]
    return 0

skill_matrix = numpy.vstack(
    tuple(
        numpy.fromiter((get_skill_level(equipment, skill) for equipment in equipment_table), int)
        for skill in skill_list
    )
)
skill_lower_bound = numpy.fromiter((desired_skills[skill] if skill in desired_skills else 0 for skill in skill_list), int)

# decos must be owned by the user
deco_matrix = numpy.vstack(
    tuple(
        numpy.fromiter((equipment["name_en"] == deco for equipment in equipment_table), int)
        for deco in deco_list
    )
)
deco_upper_bound = numpy.fromiter((owned_decos[deco] if deco in owned_decos else 0 for deco in deco_list), int)

In [7]:
from scipy import optimize

res = optimize.linprog(
    c=-defense_row,
    A_ub=numpy.concatenate((equipment_matrix, -equipment_matrix, -deco_slot_matrix, -skill_matrix, deco_matrix)),
    b_ub=numpy.concatenate((equipment_upper_bound, -equipment_lower_bound, -deco_slot_lower_bound, -skill_lower_bound, deco_upper_bound)),
    bounds=(0, 7),
    integrality=1,
)

def interpret(res, integrality = True):
    output = list()
    for count, equipment in zip(res.x, equipment_table):
        if count > 0:
            output.append((int(count) if integrality else count, equipment))
    print("\n".join(f"{count} {equipment['name_en']}" for count, equipment in output))

    skills = dict()
    for count, equipment in output:
        if equipment["skill1_name"] != "":
            if equipment["skill1_name"] not in skills:
                skills[equipment["skill1_name"]] = 0
            skills[equipment["skill1_name"]] += int(equipment["skill1_level"]) * count
        if equipment["skill2_name"] != "":
            if equipment["skill2_name"] not in skills:
                skills[equipment["skill2_name"]] = 0
            skills[equipment["skill2_name"]] += int(equipment["skill2_level"]) * count
    pprint.pprint(skills)

    print(f"defense: {sum(count * equipment['defense_base'] for count, equipment in output)}")

interpret(res)

1 Golden Lunemail β+
1 Brachydium Braces α+
1 Kulve Taroth's Fury β+
1 Tentacle Coil  γ+
1 Rimeguard Greaves  γ+
1 Capacity Charm
1 Sheath Jewel 1
3 Stonethrower Jewel 1
3 Vitality Jewel 1
1 Jumping/Physique Jewel 4
1 Wind Resist/Protection Jewel 4
{'Agitator': 2,
 'Artillery': 3,
 'Capacity Boost': 1,
 'Constitution': 1,
 'Divine Blessing': 3,
 'Evade Extender': 4,
 'Free Meal': 1,
 'Health Boost': 3,
 'Peak Performance': 3,
 'Quick Sheath': 3,
 'Recovery Speed': 2,
 'Slinger Capacity': 3,
 'Windproof': 1}
defense: 864


In [8]:
import random
res = optimize.linprog(
    c=-numpy.fromiter((random.random() for _ in defense_row), float),
    A_ub=numpy.concatenate((equipment_matrix, -equipment_matrix, -deco_slot_matrix, -skill_matrix, deco_matrix)),
    b_ub=numpy.concatenate((equipment_upper_bound, -equipment_lower_bound, -deco_slot_lower_bound, -skill_lower_bound, deco_upper_bound)),
    bounds=(0, 7),
    integrality=1,
)

interpret(res)

1 Guild Cross Circlet β
1 Dodogama Greaves β+
1 Rimeguard Mail β+
1 Brachydium Braces α+
1 Kulve Taroth's Malice α+
1 Leaping Charm III
1 Sheath Jewel 1
2 Steadfast Jewel 1
3 Stonethrower Jewel 1
2 Vitality Jewel 1
1 Sprinter Jewel 2
1 Wind Resist/Protection Jewel 4
1 Brace/Vitality Jewel 4
1 Warming Jewel 2
{'Agitator': 2,
 'Artillery': 3,
 'Capacity Boost': 1,
 'Coldproof': 1,
 'Divine Blessing': 3,
 'Evade Extender': 3,
 'Flinch Free': 1,
 'Free Meal': 1,
 'Health Boost': 3,
 'Marathon Runner': 1,
 'Peak Performance': 2,
 'Quick Sheath': 3,
 'Slinger Capacity': 3,
 'Stun Resistance': 2,
 'Windproof': 1}
defense: 682


In [9]:
possible_skills = dict()
letter = None
for new_skill in skill_list:
    for new_skill_level in range(1, 5):
        new_skill_lower_bound = numpy.fromiter((new_skill_level if new_skill == skill else 0 for skill in skill_list), int)

        res = optimize.linprog(
            c=numpy.zeros_like(defense_row),
            A_ub=numpy.concatenate((equipment_matrix, -equipment_matrix, -deco_slot_matrix, -skill_matrix, -skill_matrix, deco_matrix)),
            b_ub=numpy.concatenate((equipment_upper_bound, -equipment_lower_bound, -deco_slot_lower_bound, -skill_lower_bound, -new_skill_lower_bound, deco_upper_bound)),
            bounds=(0, 7),
            integrality=1,
        )

        if res.status == 0:
            possible_skills[new_skill] = new_skill_level

    if new_skill in possible_skills:
        if new_skill[0] != letter:
            if letter != None:
                print()
            letter = new_skill[0]
        print((new_skill, possible_skills[new_skill]), end=", ")

('Agitator', 4), ('Artillery', 3), ('Attack Boost', 3), 
('Blast Attack', 1), 
('Capacity Boost', 1), ('Cliffhanger', 1), ('Coalescence', 1), ('Constitution', 1), ('Critical Draw', 1), ('Critical Eye', 1), 
('Defense Boost', 2), ('Divine Blessing', 4), 
('Earplugs', 3), ('Evade Extender', 4), ('Evade Window', 4), 
('Flinch Free', 3), ('Focus', 1), ('Free Elem/Ammo Up', 3), ('Free Meal', 1), 
('Handicraft', 3), ('Health Boost', 4), 
('Ice Attack', 2), 
('Marathon Runner', 3), 
('Paralysis Attack', 1), ('Peak Performance', 4), ('Power Prolonger', 1), 
('Quick Sheath', 4), 
('Recovery Speed', 2), ('Recovery Up', 3), ('Resentment', 1), 
('Slinger Capacity', 4), ('Speed Eating', 3), ('Stun Resistance', 4), 
('Thunder Attack', 2), ('Thunder Resistance', 2), ('Tool Specialist', 1), 
('Wide-Range', 3), ('Windproof', 2), 